# `tensorflow.keras` sweep across pre-trained models


This notebook demonstrates the use of (1) well-known CNN architectures, (2) pretrained weights on `RadImageNet` in a multiclassification task, (3) `wandb.ai` sweeps in comparing the pre-trained architectures.

References:

* The pretrained weights published by [Mei et al (2022)](https://pubs.rsna.org/doi/10.1148/ryai.210315) in their github [link](https://github.com/BMEII-AI/RadImageNet).

* The github repository form the BAGLS team contained in this github [link](https://github.com/anki-xyz/bagls/blob/master/Utils/DataGenerator.py#L109)


In [1]:
# dev convenience
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")
import PATHS

import os
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
os.environ["WANDB_SILENT"] = "True"
os.environ["WANDB_NOTEBOOK_NAME"] = "05-pretrained-sweep.ipynb"

PROJECT_NAME = 'bagls-sh-project'
GROUP_NAME = 'pretrained-models-sweep'
METRICS_TABLE_NAME = 'metrics_table'

In [3]:
import wandb
print("W&B: ", wandb.__version__)
wandb.login()

# # manage logs
# import logging

# logger = logging.getLogger("wandb")
# logger.setLevel(logging.ERROR)

# logging.getLogger('tensorflow').disabled = True

W&B:  0.13.5


True

In [4]:
# tf loader
import tensorflow as tf
print("TF: ", tf.__version__)
from tensorflow.keras.models import load_model
from tensorflow import keras

In [5]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5085945803301618659
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10925703168
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 11950236232734223965
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 10925703168
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 1013055953366377435
physical_device_desc: "device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1"
, name: "/device:GPU:2"
device_type: "GPU"
memory_limit: 10925703168
locality {
  bus_id: 2
  numa_node: 1
  links {
    link {
      device_id: 3
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 1

2022-11-30 07:47:21.301936: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 07:47:24.330546: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1
2022-11-30 07:47:24.331441: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:1 with 10419 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1
2022-11-30 07:47:24.332115: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:2 with 10419 MB memory:  -> device: 2, name: NVIDIA GeForce GTX 1080 Ti, pci

In [6]:
import config
configs = config.nb_configs
configs.update(config.pt_configs)

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.imagenet_utils import preprocess_input

# initialize data generator
train_datagen = ImageDataGenerator(
#     preprocessing_function=preprocess_input,
    validation_split=configs["validation_split"],
    rescale=configs["rescale"],
    width_shift_range=configs["width_shift_range"],
    height_shift_range=configs["height_shift_range"],
    shear_range=configs["shear_range"],
    zoom_range=configs["zoom_range"],
    fill_mode=configs["fill_mode"],
    horizontal_flip=configs["horizontal_flip"],
    rotation_range=configs["rotation_range"],
)

val_datagen = 

test_datagen = ImageDataGenerator(
#     preprocessing_function=preprocess_input, 
    rescale=configs["rescale"],
)

In [8]:
train_dir = configs["train_dir"]
test_dir = configs["test_dir"]

batch_size = configs["batch_size"]
class_names = configs["class_names"]
interpol = configs["interpol"]
cmap = configs["cmap"]
label_mode = configs["label_mode"]
labels = configs["labels"]
image_size = configs["image_size"]


train_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="training",
)

val_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="validation",
)

test_dataset = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    shuffle=False, # do not shuffle for later evaluation, alphanum sort
)

configs.update({"val_steps": val_dataset.samples // configs["batch_size"]})

Found 52393 images belonging to 2 classes.
Found 2757 images belonging to 2 classes.
Found 3300 images belonging to 2 classes.


In [9]:
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras import losses

In [10]:
def define_model(model_arch, weights):
    conv_base = model_arch(
        weights=weights,
        include_top=configs["include_top"],
        input_shape=configs["input_shape"],
        pooling=configs["pooling"],
    )
    # freeze conv base weights
    conv_base.trainable = False
    x = conv_base.output
    
    # layers at this stage are arbitrary
    # can be subjected to hyperparam tuning
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(units=512, activation='relu')(x)
    x = keras.layers.Dropout(configs["dropout_rate"])(x)
    outputs = keras.layers.Dense(units=2, activation="softmax")(x)
    model = keras.Model(inputs=conv_base.input, outputs=outputs)
    return model

In [11]:
from interpretation import ValLog, GRADCamLogger
import utils

def train(model_name):
    GRADCAM_LAYER_NAME = config.pt_gradcam_layers[model_name]
    model_arch = config.pt_models[model_name]
    weights = config.pt_weights[model_name]
    
    tf.keras.backend.clear_session()
    
    thresh = configs["thresh"]
    metrics_dict = {
        "ACC":  metrics.CategoricalAccuracy(name="ACC"),
        "AUC-ROC": metrics.AUC(name='ROC', curve='ROC'),
        "AUC-PR": metrics.AUC(name='PR', curve='PR'),
        "TP": metrics.TruePositives(name="TP", thresholds=thresh),
        "TN": metrics.TrueNegatives(name="TN", thresholds=thresh),
        "FP": metrics.FalsePositives(name="FP", thresholds=thresh),
        "FN": metrics.FalseNegatives(name="FN", thresholds=thresh),
    }

    # opt = optimizers.Adam(learning_rate=1e-06)
    opt = optimizers.Adam()
    met = list(metrics_dict.values())

    model = define_model(model_arch, weights)
    model.compile(
        loss=losses.CategoricalCrossentropy(),
        optimizer=opt,
        metrics=met,
    )
    # initialize run
    run = wandb.init(
        project=PROJECT_NAME, 
        group=GROUP_NAME,
        name=model_name,
        job_type='train',
        config=configs, 
    )

    wandb_callback = wandb.keras.WandbCallback(
        monitor="val_ROC",
        mode="max",
        save_model=True,
        save_graph=True,
        compute_flops=True,
    )

    callbacks = [
        wandb_callback,
        ValLog(generator=val_dataset, num_log_batches=1),
        GRADCamLogger(generator=test_dataset, 
                      layer_name=GRADCAM_LAYER_NAME, num_log_batches=1)
    ]
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=configs["epochs"], 
        shuffle=True,
        callbacks=callbacks,
        verbose=0,
    )
    run.finish()
    
def evaluate(model_name):
    run = wandb.init(
        project=PROJECT_NAME, 
        group=GROUP_NAME,
        name=model_name,
        job_type="inference", 
        config=configs, 
    )
    model_at = run.use_artifact("model-" + model_name + ":latest")
    model_dir = model_at.download()
    best_model = keras.models.load_model(model_dir)
    
    metrics_results = best_model.evaluate(test_dataset)
    metrics_results = dict(zip(["loss"] + list(metrics_dict.keys()), 
                               metrics_results))
    tp, fp, tn, fn = (metrics_results["TP"], metrics_results["FP"], 
                      metrics_results["TN"], metrics_results["FN"])

    add_metrics = {
        "SENSITIVITY": utils.get_sensitivity(tp, fp, tn, fn),
        "SPECIFICTY": utils.get_specificity(tp, fp, tn, fn),
        "PPV": utils.get_ppv(tp, fp, tn, fn),
        "NPV": utils.get_npv(tp, fp, tn, fn),
        "F1" : utils.get_fbeta(tp, fp, tn, fn, beta=1),
    }
    metrics_results.update(add_metrics)

    print(f"Metrics: \n", metrics_results)

    columns = list(metrics_results.keys())
    metrics_table = wandb.Table(columns=columns)
    metrics_table.add_data(*metrics_results.values())
    wandb.run.log({METRICS_TABLE_NAME : metrics_table})

    # add logging of confusion matrix image from matplotlib

    # get preds
    trained_preds = best_model.predict(test_dataset)
    run.finish()

In [ ]:
for model_name in config.pt_models.keys():
    print(model_name)
    train(model_name)
    evaluate(model_name)

ResNet50


2022-11-30 07:47:51.994588: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1
2022-11-30 07:47:51.995580: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10419 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1
2022-11-30 07:47:51.996323: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 10419 MB memory:  -> device: 2, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1
2022-11-30 07:47:51.997119: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 10419 MB memory:  -> device: 3, name: NVIDIA GeForce

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


2022-11-30 07:48:10.474581: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-11-30 07:48:17.775033: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8400
/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
2022-11-30 08:03:20.732381: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221130_074753-ls0197dz/files/model-best/assets


...

/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221130_074753-ls0197dz/files/model-best/assets


/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221130_074753-ls0197dz/files/model-best/assets


/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


## End